In [1]:
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.datasets import mnist
 
from tensorflow.keras.utils import to_categorical

from tensorflow.keras import utils

from tensorflow.keras.preprocessing import image

from tensorflow.keras.applications.resnet50 import preprocess_input

from sklearn.model_selection import train_test_split

from keras.layers import Input, Flatten, Dense

from keras.models import Model

from tensorflow.keras.optimizers import Adam

import os

import pandas as pd

from tensorflow.keras.models import load_model

import ast

In [2]:
# Importing cnn model
cnn_model_2 = load_model('cnn_model_2.h5')

2022-10-12 17:55:00.279866: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Predicting Test Data:

In [3]:
data_path = './extracted_pics'

In [4]:
data_dir = os.listdir(data_path)

In [16]:
# importing pictures and adding them into a list

img_data_list = []
image_paths = []

for img in data_dir:
    img_path = data_path + '/' +img
    try:
        img = image.load_img(img_path, target_size= (28,28))
        x = image.img_to_array(img)
        x = preprocess_input(x)
        img_data_list.append(x)
        image_paths.append(img_path)
    except:
        continue

In [17]:
img_data = np.array(img_data_list)

In [18]:
np.save('./data/current_sq_data.npy', img_data)

In [19]:
img_data = np.load('./data/current_sq_data.npy')

In [20]:
x = img_data

# Predicting on test Data:

In [21]:
non_vegetarian_images = []

In [22]:
places_images = []

In [23]:
vegetarian_images = []

In [24]:
for i, im in enumerate(x):
    test_img = np.expand_dims(im, axis=0)
    single_pred = cnn_model_2.predict(test_img,verbose=0)
    if np.argmax(single_pred) == 0:
        vegetarian_images.append(image_paths[i])
    elif np.argmax(single_pred)==1:
        non_vegetarian_images.append(image_paths[i])
    else:
        places_images.append(image_paths[i])

In [25]:
places_df = pd.read_csv('./data/places_df.csv')

In [26]:
places_df[places_df['name']=='Kana Sushi']

,name,address,geometry,phone_number,website,total_user_ratings,reviews
31,Kana Sushi,"7020 Old Keene Mill Rd, Springfield, VA 22150,...","{'lat': 38.7793617, 'lng': -77.18718659999999}",+1 703-569-3073,http://kana-sushi.com/,425.0,"[{'author_name': 'Yerlan Jumabekov', 'author_u..."


In [42]:
ast.literal_eval(places_df['geometry'][0])['lat']

38.7775425

In [43]:
places_df['lat'] = places_df['geometry'].apply(lambda x: ast.literal_eval(x)['lat'])

In [44]:
places_df['lng'] = places_df['geometry'].apply(lambda x: ast.literal_eval(x)['lng'])

In [45]:
def text_extract(txt_list):
    text = ''
    txt = ast.literal_eval(txt_list)
    for each in txt:
        text += each['text']
    return text
    

In [46]:
places_df['text_extract'] = places_df['reviews'].apply(text_extract)

In [47]:
# Creating vegetarian Images dataframe
images_df0 = pd.DataFrame({'image_path': vegetarian_images})

In [48]:
images_df0['image_path'][0].split('/')[2].split('+')[0]

'Pho Ngon'

In [49]:
# Adding restaurant name to images_df
images_df0['name'] = images_df0['image_path'].apply(lambda x: x.split('/')[2].split('+')[0])

In [50]:
# Merging places_df with images_df
vegetarian_places = places_df.merge(images_df0, on='name')

In [51]:
# Cleaning geometry column
vegetarian_places['geometry'] = vegetarian_places['geometry'].apply(lambda x: ast.literal_eval(x))

In [52]:
# Cleaning reviews column
vegetarian_places['reviews'] = vegetarian_places['reviews'].apply(lambda x: ast.literal_eval(x))

In [53]:
def review_table(name):
    
    reviews_df = pd.DataFrame({'author':[],'rating':[],'time':[] ,'text':[]})
    reviews_list = vegetarian_places[vegetarian_places['name']== name].reviews.iloc[0]
    for each in reviews_list:
        author = each['author_name']
        rating = each['rating']
        time = each['relative_time_description']
        text = each['text']
        reviews_df.loc[len(reviews_df)] = [author, rating, time, text]
    
    return reviews_df

In [54]:
# place_review = review_table('Yard House')

In [55]:
# place_review

,image_path,name
0,./extracted_pics/Pho Ngon+2myimage.jpg,Pho Ngon
1,./extracted_pics/Karahi Kabob+7myimage.jpg,Karahi Kabob
2,./extracted_pics/Kumo Asian Bistro+7myimage.jpg,Kumo Asian Bistro
3,./extracted_pics/Saigon City Restaurant+5myima...,Saigon City Restaurant
4,./extracted_pics/Mandarin Chinese Halal Restau...,Mandarin Chinese Halal Restaurant
5,./extracted_pics/Romano's Macaroni Grill+8myim...,Romano's Macaroni Grill
6,./extracted_pics/Outback Steakhouse+1myimage.jpg,Outback Steakhouse
7,./extracted_pics/Houlihan's+6myimage.jpg,Houlihan's
8,./extracted_pics/Sarku Japan+7myimage.jpg,Sarku Japan
9,./extracted_pics/Pho Ngon+0myimage.jpg,Pho Ngon


In [59]:
# Creating non_vegetarian Images dataframe
images_df1 = pd.DataFrame({'image_path': non_vegetarian_images})

In [62]:
images_df1['name'] = images_df1['image_path'].apply(lambda x: x.split('/')[2].split('+')[0])

In [64]:
# Creating places Images dataframe
images_df2 = pd.DataFrame({'image_path': places_images})

In [65]:
images_df2['name'] = images_df2['image_path'].apply(lambda x: x.split('/')[2].split('+')[0])